In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import imgaug.augmenters as iaa
import tensorflow as tf
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
from sklearn.model_selection import train_test_split
import numpy as np
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merging import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

path_to_dataset = "/content/drive/MyDrive/Polito/MLinAP/RECHERCHE-015.npy"
path_to_labels = "/content/drive/MyDrive/Polito/MLinAP/RECHERCHE-015_label.npy"

# Data preprocessing

In [ ]:
def data_augment():
    return iaa.Sequential([
        iaa.Dropout((0, 0.05)),  # Remove random pixel
        iaa.Affine(rotate=(-30, 30)),  # Rotate between -30 and 30 degreed
        iaa.Fliplr(0.5),  # Flip with 0.5 probability
        iaa.Crop(percent=(0, 0.2), keep_size=True),  # Random crop
        # Add -50 to 50 to the brightness-related channels of each image
        iaa.WithBrightnessChannels(iaa.Add((-50, 50))),
        # Change images to grayscale and overlay them with the original image by varying strengths, effectively removing 0 to 50% of the color
        iaa.Grayscale(alpha=(0.0, 0.5)),
        # Add random value to each pixel
        iaa.GammaContrast((0.5, 2.0), per_channel=True),
        # Local distortions of images by moving points around
        iaa.PiecewiseAffine(scale=(0.01, 0.1)),
    ], random_order=True)

In [ ]:
def process_data(image, label):
    return tf.cast(image, tf.float32)/255, tf.one_hot(label, 2, name="label", axis=-1)
    #return tf.cast(image, tf.float32)/255, label
#Il one_hot (one hot encoding) è necessario se il modello è compilato con 'loss="categorical_crossentropy"'
#Nel nostro caso, vista la classificazione binaria, si può compilare il modello con 'loss="binary_crossentropy"'
#Questa NON richiede la one hot encoding!

In [ ]:
def data_aug_impl(dataset, image_train, label_train):
    da = data_augment()
    segmented_label_train = [SegmentationMapsOnImage(
        label, shape=dataset[1].shape) for label in label_train]
    image_train_copy = image_train.copy()
    for _ in range(1):
        augmented_images, augmented_labels = da(
            images=image_train_copy, segmentation_maps=segmented_label_train)
        image_train = np.append(image_train, augmented_images, axis=0)
        label_train = np.append(label_train, np.array(
            [label.get_arr() for label in augmented_labels]), axis=0)

    return image_train, label_train

In [ ]:
def generate_train_data_tensor(image_train, label_train):
    train_data = tf.data.Dataset.from_tensor_slices((image_train, label_train))
    train_data = train_data.map(
        process_data, num_parallel_calls=tf.data.AUTOTUNE)
    train_data = train_data.cache()
    train_data = train_data.shuffle(100)
    train_data = train_data.batch(32)
    train_data = train_data.prefetch(tf.data.AUTOTUNE)
    return train_data

In [ ]:
dataset = np.load(path_to_dataset)[0:10]
labels = np.load(path_to_labels)[0:10]
image_train, image_test, label_train, label_test = train_test_split(
    dataset, labels, test_size=0.25, random_state=42)
#image_train, label_train = data_aug_impl(dataset, image_train, label_train)
train_data = generate_train_data_tensor(image_train, label_train)

In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 512, 512, 2), dtype=tf.float32, name=None))>

# U-net5

In [ ]:
class Unet(Model):

  def __init__(self, dropout_value= 0.3, num_classes = 2, input_shape = (512, 512, 3)):
    super().__init__()
    vgg19 = tf.keras.applications.vgg19.VGG19(
        include_top=False,   # Exclusion of the last 3 layers
        weights='imagenet',
        # input_tensor=None,
        input_shape=input_shape,
        pooling='max',
        classes=num_classes,
        classifier_activation='relu'
    )
    #for layer in vgg19.layers:
    #  layer.trainable = False
    #Block 1
    self.b1c1 = vgg19.get_layer('block1_conv1')
    self.b1c2 = vgg19.get_layer('block1_conv2')
    #Block 2
    self.b2p = vgg19.get_layer('block1_pool')
    self.b2c1 = vgg19.get_layer('block2_conv1')
    self.b2c2 = vgg19.get_layer('block2_conv2')
    #Block 3
    self.b3p = vgg19.get_layer('block2_pool')
    self.b3c1 = vgg19.get_layer('block3_conv1')
    self.b3c2 = vgg19.get_layer('block3_conv2')
    #Block 4
    self.b4p = vgg19.get_layer('block3_pool')
    self.b4c1 = vgg19.get_layer('block4_conv1')
    self.b4c2 = vgg19.get_layer('block4_conv2')
    #Block 5
    self.b5p = vgg19.get_layer('block4_pool')
    self.b5c1 = vgg19.get_layer('block5_conv1')
    self.b5c2 = vgg19.get_layer('block5_conv2')
    #Block 6
    #self.b6d1 = Dropout(dropout_value) #Controllando meglio su internet, sembrerebbe che il dropout non è presente nativamente su unet, ma è possibile inserirlo qualora si osservi dell'overfitting
    self.b6p = vgg19.get_layer('block5_pool')
    self.b6c1 = Conv2D(filters = 1024, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b6c2 = Conv2D(filters = 1024, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    #self.b6d2 = Dropout(dropout_value)
    self.b6u = Conv2DTranspose(512, (3, 3), activation = "relu", strides = (2, 2), padding = 'same')
    #Block 7
    #After concatenate
    self.b7c1 = Conv2D(filters = 512, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b7c2 = Conv2D(filters = 512, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b7u = Conv2DTranspose(512, (3, 3), activation = "relu", strides = (2, 2), padding = 'same')
    #Block 8
    #After concatenate
    self.b8c1 = Conv2D(filters = 512, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b8c2 = Conv2D(filters = 512, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b8u = Conv2DTranspose(256, (3, 3), activation = "relu", strides = (2, 2), padding = 'same')
    #Block 9
    #After concatenate
    self.b9c1 = Conv2D(filters = 256, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b9c2 = Conv2D(filters = 256, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b9u = Conv2DTranspose(128, (3, 3), activation = "relu", strides = (2, 2), padding = 'same')
    #Block 10
    #After concatenate
    self.b10c1 = Conv2D(filters = 128, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b10c2 = Conv2D(filters = 128, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b10u = Conv2DTranspose(64, (3, 3), activation = "relu", strides = (2, 2), padding = 'same')
    #Block 11
    #After concatenate
    self.b11c1 = Conv2D(filters = 64, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b11c2 = Conv2D(filters = 64, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b11c3 = Conv2D(filters = 64, activation='relu', kernel_size = (3, 3), kernel_initializer = 'he_normal', padding = 'same')
    self.b11s = Conv2D(2, (1, 1), activation='softmax')

  def call(self, input):
    #Block 1
    r1 = self.b1c1(input)
    r1 = self.b1c2(r1)
    #Block 2
    r2 = self.b2p(r1)
    r2 = self.b2c1(r2)
    r2 = self.b2c2(r2)
    #Block 3
    r3 = self.b3p(r2)
    r3 = self.b3c1(r3)
    r3 = self.b3c2(r3)
    #Block 4
    r4 = self.b4p(r3)
    r4 = self.b4c1(r4)
    r4 = self.b4c2(r4)
    #Block 5
    r5 = self.b5p(r4)
    r5 = self.b5c1(r5)
    r5 = self.b5c2(r5)
    #Block 6
    #r6 = self.b6d1(r5)
    r6 = self.b6p(r5)
    r6 = self.b6c1(r6)
    r6 = self.b6c2(r6)
    #r6 = self.b6d2(r6)
    r6 = self.b6u(r6)
    #Block 7
    r7 = concatenate([r6, r5]) 
    r7 = self.b7c1(r7)
    r7 = self.b7c2(r7)
    r7 = self.b7u(r7)
    #Block 8
    r8 = concatenate([r7, r4]) 
    r8 = self.b8c1(r8)
    r8 = self.b8c2(r8)
    r8 = self.b8u(r8)
    #Block 9
    r9 = concatenate([r8, r3]) 
    r9 = self.b9c1(r9)
    r9 = self.b9c2(r9)
    r9 = self.b9u(r9)
    #Block 10
    r10 = concatenate([r9, r2]) 
    r10 = self.b10c1(r10)
    r10 = self.b10c2(r10)
    r10 = self.b10u(r10)
    #Block 11
    r11 = concatenate([r10, r1]) 
    r11 = self.b11c1(r11)
    r11 = self.b11c2(r11)
    r11 = self.b11c3(r11)
    out = self.b11s(r11)
    return out 
        

unet = Unet()
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
unet.compile(optimizer=optimizer,
               loss="binary_crossentropy", metrics=["accuracy"])
history = unet.fit(train_data, epochs=3)

Epoch 1/3
1/1 [==============================] - 41s 41s/step - loss: 0.5117 - accuracy: 0.9168
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 0.6338 - accuracy: 0.9187
Epoch 3/3
1/1 [==============================] - 2s 2s/step - loss: 0.4667 - accuracy: 0.9187


In [ ]:
unet = Unet()
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
unet.compile(optimizer=optimizer,
               loss="binary_crossentropy", metrics=["accuracy"])
history = unet.fit(train_data, epochs=3)

In [ ]:
history = unet.fit(train_data, epochs=3)

Epoch 1/3
